In [1]:
import requests
import datetime as dt
import time
import pytz
import json
import pandas as pd
pd.set_option('display.max_columns', None)

In [5]:
def get_exir_data_by_url(url):

    response = requests.get(url)
    result = response.json()

    return result


def get_cryptocurrency_historic_data(crypto_symbol, from_timestamp, to_timestamp):
    
    url = f'https://api.exir.io/v2/chart?symbol={crypto_symbol}&resolution=15&from={from_timestamp}&to={to_timestamp}'
    raw_data = get_exir_data_by_url(url)
    print(url)
    
    return raw_data

def transform_cryptocurrency_historic_data(raw_data, crypto_id):
    
    # change to dataframe
    df = pd.DataFrame(raw_data)

    # get interest data
    df['Crypto_Id'] = crypto_id
    df['Datetime_UTC'] = pd.to_datetime(df['time'])
    df['Datetime_Local'] = df['Datetime_UTC'].dt.tz_convert(pytz.timezone('Asia/Tehran'))
    df['Value'] = df['close']
    df['Previous_Value'] = df['open']

    # remove extra data
    df = df[['Crypto_Id', 'Datetime_UTC', 'Datetime_Local', 'Value', 'Previous_Value']]

    return df

In [11]:
to_timestamp

'1716567880'

In [12]:
from_time = dt.datetime(2024, 3, 1, tzinfo= pytz.timezone('Asia/Tehran'))
from_timestamp = str(int(from_time.timestamp()))
to_time = dt.datetime.now()
to_timestamp = str(int(to_time.timestamp()))

crypto_metadata = {
    'teter': 'usdt-irt',
    'bitcoin': 'btc-irt'
}

raw_data = get_cryptocurrency_historic_data('usdt-irt', from_timestamp, to_timestamp)
# df = transform_cryptocurrency_historic_data(raw_data, 'teter')

https://api.exir.io/v2/chart?symbol=usdt-irt&resolution=15&from=1709238840&to=1716568072


In [20]:
from_timestamp

'1740774840'

In [21]:
to_timestamp

'1716567573'

In [19]:
raw_data

[]

In [12]:
df = transform_cryptocurrency_historic_data(raw_data, 'teter')

In [15]:
pd.DataFrame(raw_data).columns

Index(['time', 'close', 'high', 'low', 'open', 'symbol', 'volume'], dtype='object')

In [4]:
df = pd.DataFrame(raw_data)

In [6]:
import pytz

In [10]:
df.iloc[0]['time']

'2024-02-29T20:30:00.000Z'

In [14]:
df['time2'] = pd.to_datetime(df['time'])
df['time3'] = df['time2'].dt.tz_convert(pytz.timezone('Asia/Tehran'))

In [15]:
df

,time,close,high,low,open,symbol,volume,time2,time3
0,2024-02-29T20:30:00.000Z,58779,58779,58750,58750,usdt-irt,958.33,2024-02-29 20:30:00+00:00,2024-03-01 00:00:00+03:30
1,2024-02-29T21:00:00.000Z,58779,58779,58690,58690,usdt-irt,276.10,2024-02-29 21:00:00+00:00,2024-03-01 00:30:00+03:30
2,2024-02-29T21:15:00.000Z,58779,58779,58629,58734,usdt-irt,633.17,2024-02-29 21:15:00+00:00,2024-03-01 00:45:00+03:30
3,2024-02-29T21:30:00.000Z,58700,58700,58700,58700,usdt-irt,6474.93,2024-02-29 21:30:00+00:00,2024-03-01 01:00:00+03:30
4,2024-02-29T21:45:00.000Z,58550,58700,58400,58700,usdt-irt,3515.83,2024-02-29 21:45:00+00:00,2024-03-01 01:15:00+03:30
...,...,...,...,...,...,...,...,...,...
6839,2024-05-24T14:45:00.000Z,58164,58164,57832,57886,usdt-irt,233.83,2024-05-24 14:45:00+00:00,2024-05-24 18:15:00+03:30
6840,2024-05-24T15:00:00.000Z,57703,58164,57703,58164,usdt-irt,1179.39,2024-05-24 15:00:00+00:00,2024-05-24 18:30:00+03:30
6841,2024-05-24T15:15:00.000Z,57672,57703,57643,57703,usdt-irt,6136.00,2024-05-24 15:15:00+00:00,2024-05-24 18:45:00+03:30
6842,2024-05-24T15:30:00.000Z,57672,57672,57644,57644,usdt-irt,2307.03,2024-05-24 15:30:00+00:00,2024-05-24 19:00:00+03:30
